In [1]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
import dill
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder
import os

In [2]:
with open('read_file.pickle', 'rb') as f: 
    load_data = dill.load(f)


df_test = load_data('test.csv')

In [3]:
with open('clean_data.pickle', 'rb') as f:
    clean_data = dill.load(f)

df_test = clean_data(df_test)

In [4]:
df_test.isna().sum().sum()

np.int64(0)

In [5]:
with open('encode_data.pickle', 'rb') as f:
    encode_data = dill.load(f)


# Get only categorical features:
categorical_cols = df_test.select_dtypes(include=['object']).columns.tolist()

df_test = encode_data(df_test, 'SalePrice', categorical_cols, train=False)


In [8]:
with open('predict_model.pickle', 'rb') as f:
    pred = dill.load(f)

with open('trained_model.pickle', 'rb') as f: 
    model = dill.load(f)

y_new_pred = pred(df_test, model, features = ['Id'])

In [9]:
y_new_pred

array([116399.07410631, 137060.79750037, 165400.4519401 , ...,
       174601.25179703, 113716.79139851, 241043.26545738])